classifiers: nn.crossentropyloss = -log-likelihood --- use for logp(y) and -logq(y/x) for support <br>
kl-div: <br>
reconstr-loss: set reduction to none and then take mean of losses per image in the total batch. This gives reconstr-loss per image for further computation<br> 

In [1]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd
import os
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from PIL.Image import LANCZOS

In [2]:
torch.cuda.is_available()

True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [5]:
from data.taskers import gen_tasks
from src.zoo.archs import ConvBase
from src.zoo.archs import TADCEncoder
from src.zoo.delpo_utils import setup

In [11]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../dataset/mini_imagenet', mode='train', n_ways=5, k_shots=5, q_shots=10, image_transforms=None)

In [125]:
classes = list(range(1623))
random.shuffle(classes)
image_transforms = transforms.Compose([transforms.Resize(28, interpolation=LANCZOS),
                                                    transforms.ToTensor(),
                                                    lambda x: 1.0 - x,
                                                ])
train_tasks = gen_tasks('omniglot', '/home/anuj/Desktop/Work/TU_Delft/research/implement/dataset/omniglot', image_transforms=image_transforms, n_ways=5, k_shots=5, q_shots=15, classes=classes[:1100])

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [12]:
ttask = train_tasks.sample()

# Gaussian Kernel Smoothing

In [26]:
n = 5*(5 + 15)
alpha = 0.1
ttask[0].shape

torch.Size([100, 3, 84, 84])

In [19]:
generic_emb = ConvBase(3, 32, 4, True, (2,2))
common = generic_emb(ttask[0])
common.shape

torch.Size([100, 32, 5, 5])

In [23]:
G = common.reshape(n, -1)
A = torch.cdist(G, G, p=2) ** 2
A = A / A.var() # Normalized adjacency matrix
A.shape

torch.Size([100, 100])

In [30]:
D = A.sum(dim=1)
D = torch.diag(D.pow(-0.5))
D

tensor([[0.6438, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5844, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6538,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.5670, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.6119, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.5660]],
       grad_fn=<DiagBackward>)

In [35]:
L = torch.matmul(torch.matmul(D, A), D)
P = torch.linalg.inv(torch.eye(n,n) - alpha * L)
P

tensor([[1.0001e+00, 1.2817e-03, 7.5209e-04,  ..., 1.3999e-03, 1.1779e-03,
         1.2399e-03],
        [1.2817e-03, 1.0001e+00, 1.3362e-03,  ..., 1.0192e-03, 1.1325e-03,
         9.8987e-04],
        [7.5209e-04, 1.3362e-03, 1.0001e+00,  ..., 1.4542e-03, 1.1138e-03,
         1.4934e-03],
        ...,
        [1.3999e-03, 1.0192e-03, 1.4542e-03,  ..., 1.0001e+00, 9.6673e-04,
         1.1531e-03],
        [1.1779e-03, 1.1325e-03, 1.1138e-03,  ..., 9.6673e-04, 1.0001e+00,
         1.2561e-03],
        [1.2399e-03, 9.8987e-04, 1.4934e-03,  ..., 1.1531e-03, 1.2561e-03,
         1.0001e+00]], grad_fn=<LinalgInvExBackward>)

In [36]:
class Kernel_Smoothing(nn.Module):
    
    def __init__(self):
        super(Kernel_Smoothing, self).__init__()

        self.n = 5*(5 + 15)
        self.alpha = 0.1

    def forward(self, x):
        x = ConvBase(3, 32, 4, True, (2,2))(x)
        G = x.reshape(self.n, -1)
        A = torch.cdist(G, G, p=2) ** 2
        A = A / A.var() # Normalized adjacency matrix
        D = torch.diag(A.sum(dim=1).pow(-0.5))
        L = torch.matmul(torch.matmul(D, A), D) # Laplacian Matrix
        P = torch.linalg.inv(torch.eye(self.n, self.n) - self.alpha * L) # Propagator Matrix
        x = torch.matmul(P, x.reshape(self.n, -1))

        return x

In [37]:
model = Kernel_Smoothing()
x = model(ttask[0])
x

tensor([[0.0756, 0.0594, 0.0413,  ..., 0.0185, 0.0437, 0.0110],
        [0.0534, 0.0178, 0.0578,  ..., 0.0203, 0.0255, 0.0122],
        [0.1110, 0.0611, 0.0659,  ..., 0.0902, 0.0795, 0.0098],
        ...,
        [0.5959, 0.1115, 0.2839,  ..., 0.0253, 0.2233, 0.1762],
        [0.2493, 0.1906, 0.0172,  ..., 0.1330, 0.1448, 0.1637],
        [0.0572, 0.2228, 0.1980,  ..., 0.0225, 0.1649, 0.2338]],
       grad_fn=<MmBackward>)

# Convolution:

In [13]:
n = 5*(5 + 10)

In [14]:
ttask[0].shape

torch.Size([75, 3, 84, 84])

In [15]:
generic_emb = ConvBase(3, 32, 4, True, (2,2))
common = generic_emb(ttask[0])
common.shape

torch.Size([75, 32, 5, 5])

In [16]:
w = 5
h = 5
c = 32

# TAFE 

In [77]:
net = common
net.shape

torch.Size([75, 32, 5, 5])

In [78]:
input = net.permute(1,0,2,3)
input.shape

torch.Size([32, 75, 5, 5])

In [79]:
# Inter-Image convolutional comparisons as feature extraction 
fe = nn.Conv2d(in_channels=n, out_channels=32, kernel_size=(1,1), stride=(1,1), padding='valid', bias=False)
input = fe(input)
input.shape

torch.Size([32, 32, 5, 5])

In [80]:
# Preparing Q, K and V
f_k = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=(1,1), stride=(1,1), padding='valid', bias=False)
f_v = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=(1,1), stride=(1,1), padding='valid', bias=False)
f_q = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=(1,1), stride=(1,1), padding='valid', bias=False)

input_q = f_q(input)
input_k = f_k(input)
input_v = f_v(input)

input_q = input_q.squeeze()
input_k = input_k.squeeze()
input_v = input_v.squeeze()

In [81]:
input_q.shape

torch.Size([32, 5, 5])

In [82]:
# Attention Module
input_q = input_q.reshape(input_q.shape[0], input_q.shape[1]*input_q.shape[2])
input_k = input_k.reshape(input_k.shape[0], input_k.shape[1]*input_k.shape[2])
input_v = input_v.reshape(input_v.shape[0], input_v.shape[1]*input_v.shape[2])

weights = torch.mm(input_q, input_k.transpose(0,1))
sm = nn.Softmax(dim=-1)
weights = sm(weights)
output = torch.mm(weights, input_v)
output.shape

torch.Size([32, 25])

In [83]:
# Transductive Mask transformed input
output = output.reshape(-1, 5, 5)
net = net * output
net.shape

torch.Size([75, 32, 5, 5])

## Pytorch Style Convolution

In [11]:
net = common
net.shape

torch.Size([100, 32, 5, 5])

In [12]:
input = net.permute(2,3,0,1)
input = input.reshape(input.shape[0] * input.shape[1], input.shape[2], input.shape[3]).unsqueeze(dim=1)
input.shape

torch.Size([25, 1, 100, 32])

In [13]:
# P
p = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(n,1), stride=(1,1), padding='valid', bias=False)
p.weight.data.fill_(2)
input = p(input)
input.shape

torch.Size([25, 64, 1, 32])

In [14]:
# Z
z = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(1,1), stride=(1,1), padding='valid', bias=False)
z.weight.data.fill_(3)
input = z(input)
input.shape

torch.Size([25, 32, 1, 32])

In [15]:
# F
f = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=(1,1), stride=(1,1), padding='valid', bias=False)
f.weight.data.fill_(0.314)
input = f(input)
input.shape

torch.Size([25, 1, 1, 32])

In [16]:
input = input.squeeze().transpose(0,1).reshape(-1, 5,5)
#input = input.reshape(5,5,-1).transpose(0,1)
input.shape

torch.Size([32, 5, 5])

In [17]:
a = net * input
a.shape

torch.Size([100, 32, 5, 5])

## Original

In [61]:
common.shape

torch.Size([100, 32, 5, 5])

In [57]:
net = common.reshape(n, h, w, -1)
net.shape

torch.Size([100, 5, 5, 32])

In [58]:
input = torch.transpose(net, 0, 2)
input = input.reshape(input.shape[0]*input.shape[1], 1, n, input.shape[3])
input.shape

torch.Size([25, 1, 100, 32])

In [13]:
# P
p = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(n,1), stride=(1,1), padding='valid', bias=False)
p.weight.data.fill_(2)
input = p(input)
input.shape

torch.Size([25, 64, 1, 32])

In [14]:
# Z
z = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(1,1), stride=(1,1), padding='valid', bias=False)
z.weight.data.fill_(3)
input = z(input)
input.shape

torch.Size([25, 32, 1, 32])

In [15]:
# F
f = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=(1,1), stride=(1,1), padding='valid', bias=False)
f.weight.data.fill_(0.314)
input = f(input)
input.shape

torch.Size([25, 1, 1, 32])

In [16]:
#input = input.reshape(5,5,-1)
input = input.reshape(5,5,-1).transpose(0,1)
input.shape

torch.Size([5, 5, 32])

In [17]:
a = net * input
a.shape

torch.Size([100, 5, 5, 32])

# Custom Implementation:

In [18]:
common.shape

torch.Size([100, 32, 5, 5])

In [19]:
G = common.reshape(n, -1)
G = G.transpose(1,0)
G = torch.unsqueeze(G, dim=0)
G.shape

torch.Size([1, 800, 100])

In [20]:
# P
p = torch.full((64, 1, n), 2, dtype=float, requires_grad=True)
G = G * p
G = G.sum(dim=-1).transpose(0,1).unsqueeze(dim=0)
G.shape

torch.Size([1, 800, 64])

In [21]:
# Z
z = torch.full((32, 1, G.shape[-1]), 3, dtype=float, requires_grad=True)
G = G * z
G = G.sum(dim=-1).transpose(0,1).unsqueeze(dim=0)
G.shape

torch.Size([1, 800, 32])

In [22]:
# F
f = torch.full((1, 1, G.shape[-1]), 0.314, dtype=float, requires_grad=True)
G = G * f
G = G.sum(dim=-1).transpose(0,1).unsqueeze(dim=0)
G.shape

torch.Size([1, 800, 1])

### Verifying with Convolution - 'Original'

In [23]:
G = G.squeeze(dim=0).reshape(5,5,-1)
G_dash = common.reshape(n, h, w, -1) * G
G_dash.shape

torch.Size([100, 5, 5, 32])

### Verifying with Pytorch Style Conv

In [23]:
G = G.squeeze().reshape(-1, 5, 5)
G_dash = common * G
G_dash.shape

torch.Size([100, 32, 5, 5])

In [24]:
G

tensor([[[135724.0339, 134871.5002, 156128.3977, 141662.0127, 113862.4311],
         [ 72774.5083,  78557.0485,  93187.9452,  92270.0155,  68308.9334],
         [ 71446.6729,  86870.4030,  86972.0233,  92074.2429,  69011.8134],
         [ 70268.2918,  72134.9665,  84626.7943,  75014.0232,  54637.2674],
         [ 48438.4008,  58953.4071,  62103.0001,  55304.1344,  36140.5821]],

        [[ 97584.1105,  85511.9181,  88730.2170,  99824.6582, 114331.1612],
         [127839.8272, 111239.2123, 111612.6852, 118368.7284, 150921.3162],
         [128820.2795, 111827.3190, 135450.9619, 139818.6286, 160964.5483],
         [128150.9199, 125615.3622, 143903.8772, 135381.8980, 169597.4901],
         [106791.7447,  87252.1827, 110241.0059, 119055.2388, 151839.1560]],

        [[439722.8316, 323994.5233, 386152.3525, 378627.8626, 273514.2983],
         [356857.0788, 239094.3322, 331720.4332, 332729.4020, 231207.7999],
         [394676.9129, 282858.4106, 349353.9971, 352598.1782, 236379.5037],
        

In [25]:
input

tensor([[[135724.0000, 134871.3125, 156128.3438, 141661.9531, 113862.4844],
         [ 72774.5078,  78557.0859,  93187.9297,  92270.0703,  68308.8906],
         [ 71446.6406,  86870.4062,  86972.0078,  92074.2422,  69011.8594],
         [ 70268.2578,  72134.9141,  84626.7578,  75013.9922,  54637.2539],
         [ 48438.3867,  58953.4023,  62103.0000,  55304.1523,  36140.5781]],

        [[ 97584.0781,  85511.9219,  88730.1797,  99824.5781, 114331.1953],
         [127839.9375, 111239.1797, 111612.5703, 118368.7500, 150921.4531],
         [128820.3047, 111827.3047, 135450.9844, 139818.5469, 160964.5156],
         [128150.9844, 125615.5000, 143903.8125, 135381.9375, 169597.5469],
         [106791.7656,  87252.1719, 110241.0000, 119055.2344, 151839.0469]],

        [[439723.0312, 323994.3125, 386152.0938, 378627.9688, 273514.0938],
         [356857.0938, 239094.3906, 331720.6875, 332729.3750, 231207.9688],
         [394676.9688, 282858.6250, 349353.9688, 352598.2812, 236379.3906],
        

In [26]:
a

tensor([[[[5.7130e+04, 4.2011e+04, 1.1792e+05, 1.1971e+05, 5.0771e+04],
          [0.0000e+00, 0.0000e+00, 9.6368e+03, 2.0541e+04, 1.4571e+04],
          [5.0759e+03, 0.0000e+00, 1.4381e+04, 2.9205e+04, 1.3847e+04],
          [4.9974e+04, 3.2189e+04, 4.8077e+04, 4.2570e+04, 0.0000e+00],
          [5.5177e+03, 3.6733e+04, 1.2604e+04, 2.2816e+04, 5.8615e+03]],

         [[1.2386e+04, 5.0113e+03, 4.6554e+04, 6.0615e+04, 2.2984e+04],
          [9.7799e+03, 1.2731e+04, 5.3444e+03, 3.2348e+04, 4.9568e+04],
          [5.7447e+04, 2.9769e+04, 3.4073e+03, 1.6146e+04, 1.8593e+04],
          [3.2046e+04, 0.0000e+00, 0.0000e+00, 3.1664e+04, 5.2744e+04],
          [3.5075e+04, 3.0278e+04, 7.2277e+04, 1.1006e+05, 1.9711e+04]],

         [[3.7381e+05, 0.0000e+00, 4.6799e+04, 1.0356e+06, 5.8617e+05],
          [2.5143e+03, 0.0000e+00, 0.0000e+00, 4.7950e+05, 2.1042e+05],
          [0.0000e+00, 3.6081e+05, 4.5002e+05, 5.7635e+05, 2.1625e+05],
          [4.8722e+05, 3.9853e+05, 3.4745e+05, 3.3123e+05, 0

In [27]:
G_dash

tensor([[[[5.7130e+04, 4.2011e+04, 1.1792e+05, 1.1971e+05, 5.0771e+04],
          [0.0000e+00, 0.0000e+00, 9.6368e+03, 2.0541e+04, 1.4571e+04],
          [5.0759e+03, 0.0000e+00, 1.4381e+04, 2.9205e+04, 1.3847e+04],
          [4.9974e+04, 3.2189e+04, 4.8077e+04, 4.2570e+04, 0.0000e+00],
          [5.5177e+03, 3.6733e+04, 1.2604e+04, 2.2816e+04, 5.8615e+03]],

         [[1.2386e+04, 5.0113e+03, 4.6554e+04, 6.0615e+04, 2.2984e+04],
          [9.7799e+03, 1.2731e+04, 5.3444e+03, 3.2348e+04, 4.9568e+04],
          [5.7447e+04, 2.9769e+04, 3.4073e+03, 1.6146e+04, 1.8593e+04],
          [3.2046e+04, 0.0000e+00, 0.0000e+00, 3.1664e+04, 5.2744e+04],
          [3.5075e+04, 3.0278e+04, 7.2277e+04, 1.1006e+05, 1.9711e+04]],

         [[3.7381e+05, 0.0000e+00, 4.6799e+04, 1.0356e+06, 5.8617e+05],
          [2.5143e+03, 0.0000e+00, 0.0000e+00, 4.7950e+05, 2.1042e+05],
          [0.0000e+00, 3.6081e+05, 4.5002e+05, 5.7635e+05, 2.1625e+05],
          [4.8722e+05, 3.9853e+05, 3.4745e+05, 3.3123e+05, 0

## Both methods give same results (Convolution - Pytorch Style && Custom Implementation - Pytorch Style Verification), hence use the Conv2d approach

### Conclusion: Reshape doesn't care about dimensional ordering of elements. It simply 'reshapes' the tensor into the required dimensions. Permute preserves the dimensional ordering of elements and shifts around entire sub-tensors (dims) to give the desired output.

In [29]:
class Args:
    def __init__(self, n_ways, k_shots, q_shots):
        #args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s
        self.n_ways = n_ways
        self.k_shots = k_shots
        self.q_shots = q_shots

args = Args(5, 5, 15)

In [45]:
class Test(nn.Module):
    
    def __init__(self,
                 num_input_channels: int,
                 base_channel_size: int,
                 latent_dim: int,
                 dataset: str,
                 #task_adapt_fn: str,
                 args,
                 act_fn: object = nn.ReLU):
        super(Test, self).__init__()

        self.args = args
        c_hid = base_channel_size
        self.net = nn.Sequential(
                nn.Conv2d(num_input_channels, c_hid, kernel_size=3, padding=1),
                nn.BatchNorm2d(c_hid),
                act_fn(),
                nn.MaxPool2d(2),  # 28 x 28, # 42 x 42

                # nn.ZeroPad2d(conv_padding),
                nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
                nn.BatchNorm2d(c_hid),
                act_fn(),
                nn.MaxPool2d(2),  # 9x9 # 21 x 21

                # nn.ZeroPad2d(conv_padding),
                nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
                nn.BatchNorm2d(c_hid),
                act_fn(),
                nn.MaxPool2d(2),  # 3x3 # 10 x 10

                # nn.ZeroPad2d(conv_padding),
                nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
                nn.BatchNorm2d(c_hid),
                act_fn(),
                nn.MaxPool2d(2)  # 1x1 # 5 x 5
            )
        
        n = args.n_ways * (args.k_shots + args.q_shots)
        self.eaen = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(n,1), stride=(1,1), padding='valid', bias=False),
            act_fn(),                                                                                                # P
            
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(1,1), stride=(1,1), padding='valid', bias=False),
            act_fn(),                                                                                                # Z
            
            nn.Conv2d(in_channels=32, out_channels=1, kernel_size=(1,1), stride=(1,1), padding='valid', bias=False),
            act_fn(),                                                                                                # F
        )

        self.h1 = nn.Linear(c_hid*25, latent_dim) 
        self.h2 = nn.Linear(c_hid*25, latent_dim)

    def forward(self, x, update:str):
        x = self.net(x)

        # Task aware embeddings
        input = x.permute(2,3,0,1)
        input = input.reshape(input.shape[0] * input.shape[1], input.shape[2], input.shape[3]).unsqueeze(dim=1)
        input = self.eaen(input)
        input = input.squeeze().transpose(0,1).reshape(-1, 5,5)
        if update == 'inner':
            x = x[:self.args.n_ways*self.args.k_shots] * input
        elif update == 'outer':
            x = x[self.args.n_ways*self.args.k_shots:] * input
        x = nn.Flatten()(x)
        
        mu = self.h1(x)
        log_var = self.h2(x)
        return mu, log_var


In [46]:
data, labels = ttask
data, labels = data.to('cpu') / 255.0, labels.to('cpu')
total = args.n_ways * (args.k_shots + args.q_shots)
queries_index = np.zeros(total)

# Extracting the evaluation datums from the entire task set, for the meta gradient calculation
for offset in range(args.n_ways):
    queries_index[np.random.choice(
        args.k_shots+args.q_shots, args.q_shots, replace=False) + ((args.k_shots + args.q_shots)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]

In [49]:
module = Test(3, 32, 64, 'miniimagenet', args)
module(torch.cat([support, queries], dim=0), 'inner')

(tensor([[ 0.0189,  0.0154,  0.0003,  ...,  0.0005,  0.0051,  0.0295],
         [ 0.0352, -0.0039,  0.0169,  ...,  0.0093,  0.0086,  0.0320],
         [ 0.0303, -0.0042,  0.0034,  ...,  0.0123,  0.0067,  0.0341],
         ...,
         [ 0.0318,  0.0028,  0.0113,  ...,  0.0104,  0.0048,  0.0198],
         [ 0.0248,  0.0057, -0.0041,  ...,  0.0027,  0.0114,  0.0251],
         [ 0.0272,  0.0055, -0.0080,  ...,  0.0125,  0.0024,  0.0138]],
        grad_fn=<AddmmBackward>),
 tensor([[ 0.0209,  0.0066,  0.0209,  ...,  0.0270, -0.0196,  0.0106],
         [ 0.0177,  0.0023,  0.0341,  ...,  0.0188, -0.0107,  0.0161],
         [ 0.0111,  0.0022,  0.0303,  ...,  0.0193, -0.0055,  0.0190],
         ...,
         [ 0.0192,  0.0044,  0.0341,  ...,  0.0166, -0.0099,  0.0067],
         [ 0.0238,  0.0064,  0.0237,  ...,  0.0270, -0.0190,  0.0079],
         [ 0.0221,  0.0030,  0.0286,  ...,  0.0256, -0.0191,  0.0048]],
        grad_fn=<AddmmBackward>))